In [2]:
# Initialize Otter
import otter
grader = otter.Notebook("HW01.ipynb")

# Homework 1: Sampling and Data Cleaning
## Due Date: Thursday, May 26, 11:59PM

**Group members**: Dongryeong Kim, Meike Liedtke, Danik Hollatz

**IMPORTANT NOTE:** The first homework needs to be submitted individually.

## Grading

Grading is broken down into autograded answers and free response.

For autograded answers, the results of your code are compared to provided and/or hidden tests. **For autograded probability questions, the provided tests will only check that your answer is within a reasonable range.**


### Please make sure to save regularly; do not rely on autosave to save your work for you!

## Scoring Breakdown

|Question|Points|
|---|---|
|1.1|2|
|1.2|1|
|1.3|1|
|1.5|2|
|1.6|1|
|2.1|1|
|2.3|1|
|3.1|1|
|3.2|2|
|3.3|1|
|4.1|2|
|4.2.1|1|
|4.2.2|1|
|4.2.3|2|
|4.2.4|2|
|5.1|1|
|5.2|2|
|5.3.1|1|
|5.3.2|1|
|5.3.3|1|
|5.3.4|1|
|5.4|3|
|**Total**|31|

## Introduction

The outcome of the US presidential election in 2016 took many people and many pollsters by surprise. In this assignment we will carry out a simulation study / post mortem in an attempt to understand what happened. Doing such an analysis is especially important even though the 2020 federal elections have already occurred.

In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

# Default plot configurations
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,8)
plt.rcParams['figure.dpi'] = 150
sns.set()

from IPython.display import display, Latex, Markdown

In [4]:
columns = ['State' , 'Trump' ,   'Clinton' , 'TotalVoters' ]

data = [['florida' , 49.02 , 47.82 , 9419886],
    ['michigan' , 47.50 , 47.27  ,  4799284],
    ['pennsylvania' , 48.18 , 47.46 ,  6165478],
    ['wisconsin' , 47.22 , 46.45  ,  2976150]]

voted_df = pd.DataFrame(data, columns=columns)
voted_df.head()

,State,Trump,Clinton,TotalVoters
0,florida,49.02,47.82,9419886
1,michigan,47.50,47.27,4799284
2,pennsylvania,48.18,47.46,6165478
3,wisconsin,47.22,46.45,2976150


# Election Polling

Political polling is a type of public opinion polling that can at best represent a snapshot of public opinion at the particular moment in time. Voter opinion shifts from week to week, even day to day, as candidates battle it out on the campaign field.

Polls usually start with a "horse-race" question, where respondents are asked whom they would vote for in a head-to-head race if the election were tomorrow: Candidate A or Candidate B. The survey begins with this question so that the respondent is not influenced by any of the other questions asked in the survey. Some of these other questions are asked to help assess how likely is it that the respondent will vote. Other questions are asked about age, education, and sex in order to adjust the findings if one group appears overly represented in the sample.

Pollsters typically use [random digit dialing](https://en.wikipedia.org/wiki/Random_digit_dialing) to contact people.

<!-- END QUESTION -->



### How might the sampling frame differ from the population?

After the fact, many experts have studied the 2016 election results. For example, according to the American Association for Public Opinion Research (AAPOR), predictions made before the election were flawed for three key reasons:

1. voters changed their preferences a few days before the election
2. those sampled were not representative of the voting population, e.g., some said that there was an overrepresentation of college graduates in some poll samples
3. voters kept their support for Trump to themselves (hidden from the pollsters)

In Questions 1 and 2, we will do two things:

+ Question 1: We will carry out a study of the sampling error when there is no bias. In other words, we will try to compute the chance that we get the election result wrong even if we collect our sample in a manner that is completely correct. In this case, any failure of our prediction is due entirely to random chance.
+ Question 2: We will carry out a study of the sampling error when there is bias of the second type from the list above. In other words, we will try to compute the chance that we get the election result wrong if we have a small systematic bias. In this case, any failure of our prediction is due to a combination of random chance and our bias.


<!-- END QUESTION -->



### How large was the sampling error?

In some states the race was very close, and it may have been simply sampling error, i.e., random chance that the majority of the voters chosen for the sample voted for Clinton.

A 2- or 3-point polling error in Trump’s favor (typical error historically) would likely be enough to tip the Electoral College to him.

One year after the 2016 election, Nate Silver wrote in
*The Media Has A Probability Problem* that
the "media’s demand for certainty -- and its lack of statistical rigor -- is a bad match for our complex world."
FiveThirtyEight forecasted that Clinton had about a 70 percent chance of winning.

We will first carry out a simulation study to assess the impact of the sampling error on the predictions.


## The Electoral College

The US president is chosen by the Electoral College, not by the
popular vote. Each state is alotted a certain number of
electoral college votes, as a function of their population.
Whomever wins in the state gets all of the electoral college votes for that state.

There are 538 electoral college votes (hence the name of the Nate Silver's site, FiveThirtyEight).

Pollsters correctly predicted the election outcome in 46 of the 50 states.
For these 46 states Trump received 231 and Clinton received 232 electoral college votes.

The remaining 4 states accounted for a total of 75 votes, and
whichever candidate received the majority of the electoral college votes in these states would win the election.

These states were Florida, Michigan, Pennsylvania, and Wisconsin.

|State |Electoral College Votes|
| --- | --- |
|Florida | 29 |
|Michigan | 16 |
|Pennsylvania | 20 |
|Wisconsin | 10|

For Donald Trump to win the election, he had to win either:
* Florida + one (or more) other states
* Michigan, Pennsylvania, and Wisconsin


The electoral margins were very narrow in these four states, as seen below:


|State | Trump |   Clinton | Total Voters |
| --- | --- |  --- |  --- |
|Florida | 49.02 | 47.82 | 9,419,886  |
|Michigan | 47.50 | 47.27  |  4,799,284|
|Pennsylvania | 48.18 | 47.46 |  6,165,478|
|Wisconsin | 47.22 | 46.45  |  2,976,150|

Those narrow electoral margins can make it hard to predict the outcome given the sample sizes that the polls used.

---
## Simulation Study of the Sampling Error

Now that we know how people actually voted, we can carry
out a simulation study that imitates the polling.

Our ultimate goal in this problem is to understand the chance that we will incorrectly call the election for Hillary Clinton even if our sample was collected with absolutely no bias.

### Question 1

#### Part 1.1

For your convenience, the results of the vote in the four pivotal states is repeated below:

|State | Trump |   Clinton | Total Voters |
| --- | --- |  --- |  --- |
|Florida | 49.02 | 47.82 | 9,419,886  |
|Michigan | 47.50 | 47.27  |  4,799,284|
|Pennsylvania | 48.18 | 47.46 |  6,165,478|
|Wisconsin | 47.22 | 46.45  |  2,976,150|


Using the table above, write a function `draw_state_sample(N, state)` that returns a sample with replacement of N voters from the given state. Your result should be returned as a list, where the first element is the number of Trump votes, the second element is the number of Clinton votes, and the third is the number of Other votes. For example, `draw_state_sample(1500, "florida")` could return `[727, 692, 81]`. You may assume that the state name is given in all lower case.

You might find `np.random.multinomial` useful.

<!--
BEGIN QUESTION
name: q6a
points: 2
-->

In [5]:
import random

def draw_state_sample(N, state):
    voted_df['other'] = 100 - (voted_df['Trump'] + voted_df['Clinton'])
    row = voted_df.loc[voted_df['State'] == state]
    sample = np.random.multinomial(N, [row.iloc[0,1]/100, row.iloc[0,2]/100, row.iloc[0,4]/100])
    return sample

draw_state_sample(1500,'florida') 


array([745, 713,  42])

In [6]:
grader.check("q1.1")

q1.1 results: All test cases passed!

#### Part 1.2

Now, create a function `trump_advantage` that takes in a sample of votes (like the one returned by `draw_state_sample`) and returns the difference in the proportion of votes between Trump and Clinton. For example `trump_advantage([100, 60, 40])` would return `0.2`, since Trump had 50% of the votes in this sample and Clinton had 30%.

<!--
BEGIN QUESTION
name: q6b
points: 1
-->

In [7]:
def trump_advantage(voter_sample):
    #test = [100, 60,40]
    votes_sum = np.sum(voter_sample)
    prop_diff = voter_sample[0]/votes_sum - voter_sample[1]/votes_sum
    return prop_diff

   
trump_advantage(draw_state_sample(1500,'florida'))

0.06666666666666671

In [8]:
grader.check("q1.2")

q1.2 results: All test cases passed!

#### Part 1.3

Simulate Trump's advantage across 100,000 samples of 1500 voters for the state of Pennsylvania and store the results of each simulation in a list called `simulations`.

That is, `simulations[i]` should be Trump's proportion advantage for the `i+1`th simple random sample.

<!--
BEGIN QUESTION
name: q6c
points: 1
-->

In [9]:
import itertools
    
def simulate(function, param1, param2, num):
    arr = []
    for _ in itertools.repeat(None, num):
        advantage = trump_advantage(function(param1, param2))
        arr.append(advantage)
    return arr

num = 100000
simulations = simulate(draw_state_sample, 1500,'pennsylvania', num)


In [10]:
grader.check("q1.3")

q1.3 results: All test cases passed!

<!-- END QUESTION -->

#### Part 1.5

Now write a function `trump_wins(N)` that creates a sample of N voters for each of the four crucial states (Florida, Michigan, Pennsylvania, and Wisconsin) and returns 1 if Trump is predicted to win based on these samples and 0 if Trump is predicted to lose.

Recall that for Trump to win the election, he must either:
* Win the state of Florida and 1 or more other states
* Win Michigan, Pennsylvania, and Wisconsin

<!--
BEGIN QUESTION
name: q6e
manual: false
points: 2
-->

In [11]:
def trump_wins(N):
    states = ['florida','michigan','pennsylvania','wisconsin']
    votes = []
    win = []
    weighing = np.array([2,1,1,1])
    win_threshhold = 3
    for state in states:
        votes.append(draw_state_sample(N, state))
    for state in votes:
        if (state[0] > state[1]):
            win.append(True)
        else:
            win.append(False)
    
    win = np.array(win)
    return (sum(win * weighing) > 2)

trump_wins(1500)

False

In [12]:
grader.check("q1.5")

q1.5 results: All test cases passed!

#### Part 1.6

If we repeat 100,000 simulations of the election, i.e. we call `trump_wins(1500)` 100,000 times, what proportion of these simulations predict a Trump victory? Give your answer as `proportion_trump`.

This number represents the percent chance that a given sample will correctly predict Trump's victory *even if the sample was collected with absolutely no bias*.

**Note: Many laypeople, even well educated ones, assume that this number should be 1. After all, how could a non-biased sample be wrong? This is the type of incredibly important intuition we hope to develop in you throughout this class and your future data science coursework.**

<!--
BEGIN QUESTION
name: q6f
manual: false
points: 1
-->

In [13]:
def simulate2(function, parameter, num):
    arr = []
    for _ in itertools.repeat(None, num):
        win = function(parameter)
        arr.append(win)
    return arr

num = 100000
trump_win = simulate2(trump_wins, 1500, num)
proportion_trump = sum(trump_win)/num
proportion_trump

0.69415

In [14]:
grader.check("q1.6")

q1.6 results: All test cases passed!

We have just studied the sampling error, and found how
our predictions might look if there was no bias in our
sampling process.
Essentially, we assumed that the people surveyed didn't change their minds,
didn't hide who they voted for, and were representative
of those who voted on election day.

---
## Simulation Study of Selection Bias

According to [an article](https://blogs.lse.ac.uk/usappblog/2018/02/01/better-poll-sampling-would-have-cast-more-doubt-on-the-potential-for-hillary-clinton-to-win-the-2016-election/#Author) by Grotenhuis, Subramanian, Nieuwenhuis, Pelzer and Eisinga:

> In a perfect world, polls sample from the population of voters, who would state their political preference perfectly clearly and then vote accordingly.

That's the simulation study that we just performed.


It's difficult to control for every source of selection bias.
And, it's not possible to control for some of the other sources of bias.

Next we investigate the effect of small sampling bias on the polling results in these four battleground states.

Throughout this problem, we'll examine the impacts of a 0.5 percent bias in favor of Clinton in each state. Such a bias has been suggested because highly educated voters tend to be more willing to participate in polls.

### Question 2

Throughout this problem, adjust the selection of voters so that there is a 0.5% bias in favor of Clinton in each of these states.

For example, in Pennsylvania, Clinton received 47.46 percent of the votes and Trump 48.18 percent. Increase the population of Clinton voters to 47.46 + 0.5  percent and correspondingly decrease the percent of Trump voters.

#### Part 2.1

Simulate Trump's advantage across 100,000 simple random samples of 1500 voters for the state of Pennsylvania and store the results of each simulation in a list called `biased_simulations`.

That is, `biased_simulations[i]` should hold the result of the `i+1`th simulation.

That is, your answer to this problem should be just like your answer from Question 1 part 3, but now using samples that are biased as described above.

<!--
BEGIN QUESTION
name: q7a
points: 1
-->

In [15]:
def draw_biased_state_sample(N, state):
    voted_df['other'] = 100 - (voted_df['Trump'] + voted_df['Clinton'])
    row = voted_df.loc[voted_df['State'] == state]
    sample = np.random.multinomial(N, [row.iloc[0,1]/100-.005, row.iloc[0,2]/100+.005, row.iloc[0,4]/100])
    return sample

num = 100000
biased_simulations = simulate(draw_biased_state_sample, 1500, "pennsylvania", num)

In [16]:
grader.check("q2.1")

q2.1 results: All test cases passed!

<!-- END QUESTION -->

#### Part 2.3

Now perform 100,000 simulations of all four states and return the proportion of these simulations that result in a Trump victory. This is the same fraction that you computed in Question 1 Part 6, but now using your biased samples.

Give your answer as `proportion_trump_biased`.

This number represents the chance that a sample biased 0.5% in Hillary Clinton's favor will correctly predict Trump's victory. You should observe that the chance is signficantly lower than with an unbiased sample, i.e. your answer in Q1.6.

<!--
BEGIN QUESTION
name: q7d
manual: false
points: 1
-->

In [17]:
def trump_wins_biased(N):
    states = ['florida','michigan','pennsylvania','wisconsin']
    votes = []
    win = []
    weighing = np.array([2,1,1,1])
    win_threshhold = 3
    for state in states:
        votes.append(draw_biased_state_sample(N, state))
    for state in votes:
        if (state[0] > state[1]):
            win.append(True)
        else:
            win.append(False)
    
    win = np.array(win)
    return (sum(win * weighing) > 2)

trump_wins(1500)

num = 100000
trump_win_baised = simulate2(trump_wins_biased, 1500, num)
proportion_trump_biased = sum(trump_win_baised)/num
proportion_trump_biased

0.46724

In [18]:
grader.check("q2.3")

q2.3 results: All test cases passed!

# Food Safety
## Cleaning and Exploring Data with Pandas


## This Assignment

For the second part of the Homework, we will investigate restaurant food safety scores for restaurants in San Francisco. The scores and violation information have been [made available by the San Francisco Department of Public Health](https://data.sfgov.org/Health-and-Social-Services/Restaurant-Scores-LIVES-Standard/pyih-qa8i). The main goal for this assignment is to walk through the process of Data Cleaning and EDA.


As we clean and explore these data, you will gain practice with:
* Reading simple csv files and using Pandas
* Working with data at different levels of granularity
* Identifying the type of data collected, missing values, anomalies, etc.
* Exploring characteristics and distributions of individual variables

## Before You Start

For each question in the assignment, please write down your answer in the answer cell(s) right below the question.

We understand that it is helpful to have extra cells breaking down the process towards reaching your final answer. If you happen to create new cells below your answer to run codes, **NEVER** add cells between a question cell and the answer cell below it. It will cause errors when we run the autograder, and it will sometimes cause a failure to generate the PDF file.

**Important note: The local autograder tests will not be comprehensive. You can pass the automated tests in your notebook but still fail tests in the autograder.** Please be sure to check your results carefully.

Finally, unless we state otherwise, **do not use for loops or list comprehensions**. The majority of this assignment can be done using builtin commands in Pandas and numpy.  Our autograder isn't smart enough to check, but you're depriving yourself of key learning objectives if you write loops / comprehensions, and you also won't be ready for the exam.


In [25]:
import zipfile
from pathlib import Path

import os
import plotly

from IPython.display import display, Image
def display_figure_for_grader(fig):
    plotly.io.write_image(fig, 'temp.png')
    display(Image('temp.png'))

## Obtaining the Data

### File Systems and I/O



In general, we will focus on using python commands to investigate files.  However, it can sometimes be easier to use shell commands in your local operating system.  The following cells demonstrate how to do this.

In [26]:
from pathlib import Path
data_dir = Path('.')
data_dir.mkdir(exist_ok = True)
file_path = data_dir / Path('data.zip')
dest_path = file_path

After running the cell above, if you list the contents of the directory containing this notebook, you should see `data.zip`.

*Note*: The command below starts with an `!`. This tells our Jupyter notebook to pass this command to the operating system. In this case, the command is the `ls` Unix command which lists files in the current directory.

In [27]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 5241-5BB2

 C:\Users\insoy\Desktop\깍두기\Lebniz 교환학생\학기중\DataScienceFoundations\HW01 디렉터리

2022-05-24  오후 02:40    <DIR>          .
2022-05-24  오후 02:40    <DIR>          ..
2022-05-24  오후 02:17    <DIR>          .ipynb_checkpoints
2022-05-24  오후 02:40            46,643 .OTTER_LOG
2022-05-18  오전 11:31    <DIR>          data
2022-02-21  오후 03:55           609,879 data.zip
2022-05-19  오후 03:19           156,115 Dongryeong_HW01_Q3.ipynb
2022-02-21  오후 03:55             1,892 dsf_utils.py
2022-05-18  오후 08:14           113,274 DS_HW01_03.ipynb
2022-05-19  오후 01:42            74,020 DS_HW01_Q1_Meike.ipynb
2022-05-24  오후 02:40            78,256 Finalized_HW01.ipynb
2022-05-13  오후 03:27    <DIR>          tests
2022-05-18  오전 11:34    <DIR>          __pycache__
2022-05-18  오전 11:34             7,776 전자 필기장 열기.onetoc2
               8개 파일           1,087,855 바이트
               6개 디렉터리  26,560,303,104 바이트 남음


### Question 3: Loading Food Safety Data

We have data, but we don't have any specific questions about the data yet. Let's focus on understanding the structure of the data; this involves answering questions such as:

* Is the data in a standard format or encoding?
* Is the data organized in records?
* What are the fields in each record?

Let's start by looking at the contents of `data.zip`. It's not just a single file but rather a compressed directory of multiple files. We could inspect it by uncompressing it using a shell command such as `!unzip data.zip`, but in this homework we're going to do almost everything in Python for maximum portability.

## Looking Inside and Extracting the Zip Files

The following codeblocks are setup. Simply run the cells; **do not modify them**. Question 1a is where you will start to write code.

Here, we assign `my_zip` to a `zipfile.Zipfile` object representing `data.zip`, and assign `list_names` to a list of all the names of the contents in `data.zip`.

In [28]:
my_zip = zipfile.ZipFile(dest_path, 'r')
list_names = my_zip.namelist()
list_names

['data/',
 'data/bus.csv',
 'data/ins.csv',
 'data/ins2vio.csv',
 'data/vio.csv',
 'data/sf_zipcodes.json',
 'data/legend.csv']

You may notice that we did not write `zipfile.ZipFile('data.zip', ...)`. Instead, we used `zipfile.ZipFile(dest_path, ...)`. In general, we **strongly suggest having your filenames hard coded as string literals only once** in a notebook. It is very dangerous to hard code things twice because if you change one but forget to change the other, you can end up with bugs that are very hard to find.

Now we display the files' names and their sizes.

In [29]:
my_zip = zipfile.ZipFile(dest_path, 'r')
for info in my_zip.infolist():
    print('{}\t{}'.format(info.filename, info.file_size))

data/	0
data/bus.csv	665365
data/ins.csv	1860919
data/ins2vio.csv	1032799
data/vio.csv	4213
data/sf_zipcodes.json	474
data/legend.csv	120


Often when working with zipped data, we'll never unzip the actual zipfile. This saves space on our local computer. However, for this homework the files are small, so we're just going to unzip everything. This has the added benefit that you can look inside the csv files using a text editor, which might be handy for understanding the structure of the files. The cell below will unzip the csv files into a subdirectory called `data`.

In [30]:
data_dir = Path('.')
my_zip.extractall(data_dir)
!dir {data_dir / Path("data")}

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 5241-5BB2

 C:\Users\insoy\Desktop\깍두기\Lebniz 교환학생\학기중\DataScienceFoundations\HW01\data 디렉터리

2022-05-18  오전 11:31    <DIR>          .
2022-05-18  오전 11:31    <DIR>          ..
2022-05-24  오후 03:01           665,365 bus.csv
2022-05-24  오후 03:01         1,860,919 ins.csv
2022-05-24  오후 03:01         1,032,799 ins2vio.csv
2022-05-24  오후 03:01               120 legend.csv
2022-05-24  오후 03:01               474 sf_zipcodes.json
2022-05-24  오후 03:01             4,213 vio.csv
2022-05-18  오전 11:31             5,040 전자 필기장 열기.onetoc2
               7개 파일           3,568,930 바이트
               2개 디렉터리  26,560,303,104 바이트 남음


The cell above created a folder called `data`, and in it there should be five CSV files. Let's open up `legend.csv` to see its contents. To do this, click on the jupyterhub logo on the top left, then navigate to `su21/hw/hw3/data/` and click on `legend.csv`. The file will open up in another tab. You should see something that looks like:

    "Minimum_Score","Maximum_Score","Description"
    0,70,"Poor"
    71,85,"Needs Improvement"
    86,90,"Adequate"
    91,100,"Good"

In [31]:
import dsf_utils

data_dir = "./"
for f in list_names:
    if not os.path.isdir(f):
        print(dsf_utils.head(data_dir + f, 5), "\n")

['"business id column","name","address","city","state","postal_code","latitude","longitude","phone_number"\n', '"1000","HEUNG YUEN RESTAURANT","3279 22nd St","San Francisco","CA","94110","37.755282","-122.420493","-9999"\n', '"100010","ILLY CAFFE SF_PIER 39","PIER 39  K-106-B","San Francisco","CA","94133","-9999","-9999","+14154827284"\n', '"100017","AMICI\'S EAST COAST PIZZERIA","475 06th St","San Francisco","CA","94103","-9999","-9999","+14155279839"\n', '"100026","LOCAL CATERING","1566 CARROLL AVE","San Francisco","CA","94124","-9999","-9999","+14155860315"\n'] 

['"iid","date","score","type"\n', '"100010_20190329","03/29/2019 12:00:00 AM","-1","New Construction"\n', '"100010_20190403","04/03/2019 12:00:00 AM","100","Routine - Unscheduled"\n', '"100017_20190417","04/17/2019 12:00:00 AM","-1","New Ownership"\n', '"100017_20190816","08/16/2019 12:00:00 AM","91","Routine - Unscheduled"\n'] 

['"iid","vid"\n', '"97975_20190725","103124"\n', '"85986_20161011","103114"\n', '"95754_2019032

## Reading in and Verifying Data

Based on the above information, let's attempt to load `bus.csv`, `ins2vio.csv`, `ins.csv`, and `vio.csv` into pandas dataframes with the following names: `bus`, `ins2vio`, `ins`, and `vio` respectively.

*Note:* Because of character encoding issues one of the files (`bus`) will require an additional argument `encoding='ISO-8859-1'` when calling `pd.read_csv`. At some point in your future, you should read all about [character encodings](https://diveintopython3.problemsolving.io/strings.html). We won't discuss these in detail in Data Science Foundations.

In [32]:
# path to directory containing data
dsDir = Path('data')

bus = pd.read_csv(dsDir/'bus.csv', encoding='ISO-8859-1')
ins2vio = pd.read_csv(dsDir/'ins2vio.csv')
ins = pd.read_csv(dsDir/'ins.csv')
vio = pd.read_csv(dsDir/'vio.csv')

#This code is essential for the autograder to function properly. Do not edit
ins_test = ins

Now that you've read in the files, let's try some `pd.DataFrame` methods ([docs](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.html)).
Use the `DataFrame.head` method to show the top few lines of the `bus`, `ins`, and `vio` dataframes. For example, running the cell below will display the first few lines of the `bus` dataframe.

In [33]:
bus.head()

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675


To show multiple return outputs in one single cell, you can use `display()`.

In [34]:
display(bus.head())
display(ins.head())
display(vio.head())

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675


,iid,date,score,type
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup


,description,risk_category,vid
0,Consumer advisory not provided for raw or unde...,Moderate Risk,103128
1,Contaminated or adulterated food,High Risk,103108
2,Discharge from employee nose mouth or eye,Moderate Risk,103117
3,Employee eating or smoking,Moderate Risk,103118
4,Food in poor condition,Moderate Risk,103123


The `DataFrame.describe` method can also be handy for computing summaries of numeric columns of our dataframes. Try it out with each of our 4 dataframes. Below, we have used the method to give a summary of the `bus` dataframe.

In [35]:
bus.describe()

,business id column,latitude,longitude,phone_number
count,6253.000000,6253.000000,6253.000000,6.253000e+03
mean,60448.948984,-5575.337966,-5645.817699,4.701819e+09
std,36480.132445,4983.390142,4903.993683,6.667508e+09
min,19.000000,-9999.000000,-9999.000000,-9.999000e+03
25%,18399.000000,-9999.000000,-9999.000000,-9.999000e+03
50%,75685.000000,-9999.000000,-9999.000000,-9.999000e+03
75%,90886.000000,37.776494,-122.421553,1.415533e+10
max,102705.000000,37.824494,0.000000,1.415988e+10


Now, we perform some sanity checks for you to verify that the data was loaded with the correct structure. Run the following cells to load some basic utilities (you do not need to change these at all):

First, we check the basic structure of the data frames you created:

In [36]:
assert all(bus.columns == ['business id column', 'name', 'address', 'city', 'state', 'postal_code',
                           'latitude', 'longitude', 'phone_number'])
assert 6250 <= len(bus) <= 6260

assert all(ins.columns == ['iid', 'date', 'score', 'type'])
assert 26660 <= len(ins) <= 26670

assert all(vio.columns == ['description', 'risk_category', 'vid'])
assert 60 <= len(vio) <= 65

assert all(ins2vio.columns == ['iid', 'vid'])
assert 40210 <= len(ins2vio) <= 40220

Next we'll check that the statistics match what we expect. The following are hard-coded statistical summaries of the correct data.

In [37]:
bus_summary = pd.DataFrame(**{'columns': ['business id column', 'latitude', 'longitude'],
 'data': {'business id column': {'50%': 75685.0, 'max': 102705.0, 'min': 19.0},
  'latitude': {'50%': -9999.0, 'max': 37.824494, 'min': -9999.0},
  'longitude': {'50%': -9999.0,
   'max': 0.0,
   'min': -9999.0}},
 'index': ['min', '50%', 'max']})

ins_summary = pd.DataFrame(**{'columns': ['score'],
 'data': {'score': {'50%': 76.0, 'max': 100.0, 'min': -1.0}},
 'index': ['min', '50%', 'max']})

vio_summary = pd.DataFrame(**{'columns': ['vid'],
 'data': {'vid': {'50%': 103135.0, 'max': 103177.0, 'min': 103102.0}},
 'index': ['min', '50%', 'max']})

from IPython.display import display

print('What we expect from your Businesses dataframe:')
display(bus_summary)
print('What we expect from your Inspections dataframe:')
display(ins_summary)
print('What we expect from your Violations dataframe:')
display(vio_summary)

What we expect from your Businesses dataframe:


,business id column,latitude,longitude
min,19.0,-9999.000000,-9999.0
50%,75685.0,-9999.000000,-9999.0
max,102705.0,37.824494,0.0


What we expect from your Inspections dataframe:


,score
min,-1.0
50%,76.0
max,100.0


What we expect from your Violations dataframe:


,vid
min,103102.0
50%,103135.0
max,103177.0


The code below defines a testing function that we'll use to verify that your data has the same statistics as what we expect. Run these cells to define the function. The `df_allclose` function has this name because we are verifying that all of the statistics for your dataframe are close to the expected values. Why not `df_allequal`? It's a bad idea in almost all cases to compare two floating point values like 37.780435, as rounding error can cause spurious failures.

In [38]:
"""Run this cell to load this utility comparison function that we will use in various
tests below (both tests you can see and those we run internally for grading).

Do not modify the function in any way.
"""


def df_allclose(actual, desired, columns=None, rtol=5e-2):
    """Compare selected columns of two dataframes on a few summary statistics.

    Compute the min, median and max of the two dataframes on the given columns, and compare
    that they match numerically to the given relative tolerance.

    If they don't match, an AssertionError is raised (by `numpy.testing`).
    """
    # summary statistics to compare on
    stats = ['min', '50%', 'max']

    # For the desired values, we can provide a full DF with the same structure as
    # the actual data, or pre-computed summary statistics.
    # We assume a pre-computed summary was provided if columns is None. In that case,
    # `desired` *must* have the same structure as the actual's summary
    if columns is None:
        des = desired
        columns = desired.columns
    else:
        des = desired[columns].describe().loc[stats]

    # Extract summary stats from actual DF
    act = actual[columns].describe().loc[stats]

    return np.allclose(act, des, rtol)

We will now explore each file in turn, including determining its granularity and primary keys and exploring many of the variables individually. Let's begin with the businesses file, which has been read into the `bus` dataframe.

<br/><br/><br/>

---

<br/><br/><br/>

#### Part 3.1: Examining the Business Data File

From its name alone, we expect the `bus.csv` file to contain information about the restaurants. Let's investigate the granularity of this dataset.

In [39]:
bus.head()

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675


The `bus` dataframe contains a column called `business id column` which probably corresponds to a unique business id.  However, we will first rename that column to `bid` for simplicity.

**Note**: In practice we might want to do this renaming when the table is loaded but for grading purposes we will do it here.


In [40]:
bus = bus.rename(columns={"business id column": "bid"})

Examining the entries in `bus`, is the `bid` unique for each record (i.e. each row of data)? Your code should compute the answer, i.e. don't just hard code `True` or `False`.

Hint: use `value_counts()` or `unique()` to determine if the `bid` series has any duplicates. **For documentation on these methods, see [https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) and [https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html).**

<!--
BEGIN QUESTION
name: q1a
points: 1
-->

In [41]:
def isUnique(series):
    return (len(series) == len(set(series)))

    
is_bid_unique = isUnique(bus.bid)
is_bid_unique


True

In [42]:
grader.check("q3.1")

q3.1 results: All test cases passed!

<br/><br/><br/>

---


#### Part 3.2

We will now work with some important fields in `bus`.

1. Assign `top_names` to an iterable containing the top 5 most frequently used business names, from most frequent to least frequent.
2. Assign `top_addresses` to an iterable containing the top 5 addressses where businesses are located, from most popular to least popular.

Recall that "an iterable value is anything that can be passed to the built-in iter function. Iterables include sequence values such as strings and tuples, as well as other containers such as sets and dictionaries."

Hint: You may find `value_counts()` helpful.

Hint 2: You'll need to somehow get the names / addresses, NOT the counts associated with each. If you're not sure how to do this, try looking through the class notes or using a search engine. We know this is annoying but we're trying to help you build independence.

Hint 3: To check your answer, `top_names[0]` should return the string `Peet's Coffee & Tea`. It should not be a number.

<!--
BEGIN QUESTION
name: q1b
points: 2
-->

In [43]:
top_names = bus.name.value_counts().head(n = 5).index.tolist()
top_addresses = bus.address.value_counts().head(n = 5).index.tolist()
top_names, top_addresses

(["Peet's Coffee & Tea",
  'Starbucks Coffee',
  "McDonald's",
  'Jamba Juice',
  'STARBUCKS'],
 ['Off The Grid', '428 11th St', '2948 Folsom St', '3251 20th Ave', 'Pier 41'])

In [44]:
grader.check("q3.2")

q3.2 results: All test cases passed!

<br/><br/><br/>

---


#### Part 3.3

Based on the above exploration, what does each record represent?

A. "One location of a restaurant."
B. "A chain of restaurants."
C. "A city block."

Answer in the following cell. Your answer should be a string, either `"A"`, `"B"`, or `"C"`.

<!--
BEGIN QUESTION
name: q1c
points:
- 0
- 1
-->

In [45]:
# What does each record represent?  Valid answers are:
#    "One location of a restaurant."
#    "A chain of restaurants."
#    "A city block."

q1c = "A"

In [46]:
grader.check("q3.3")

q3.3 results: All test cases passed!

<br/><br/><br/>

---

<br/><br/><br/>

### Question 4: Cleaning the Business Data Postal Codes

The business data contains postal code information that we can use to aggregate the ratings over regions of the city.  Let's examine and clean the postal code field.  The postal code (sometimes also called a [ZIP code](https://en.wikipedia.org/wiki/ZIP_Code)) partitions the city into regions:

<img src="https://www.usmapguide.com/wp-content/uploads/2019/03/printable-san-francisco-zip-code-map.jpg" alt="ZIP Code Map" style="width: 600px">

<br/><br/><br/>

---


#### Part 4.1

How many restaurants are in each ZIP code?

In the cell below, create a **series** where the index is the postal code and the value is the number of records with that postal code in descending order of count. You may need to use `groupby()`, `size()`, or `value_counts()`. Do you notice any odd/invalid zip codes?

<!--
BEGIN QUESTION
name: q2a
points: 2
-->

In [47]:
# bus = pd.read_csv(dsDir/'bus.csv', encoding='ISO-8859-1')
# bus['postal_code'].value_counts()
zip_counts = bus['postal_code'].value_counts()
# print(zip_counts.to_string())

In [48]:
grader.check("q4.1")

q4.1 results: All test cases passed!

<br/><br/><br/>

---

#### Part 4.2

In question 2a we noticed a large number of potentially invalid ZIP codes (e.g., "Ca").  These are likely due to data entry errors.  To get a better understanding of the potential errors in the zip codes we will:

1. Import a list of valid San Francisco ZIP codes by using `pd.read_json` to load the file `data/sf_zipcodes.json` and ultimately create a **series** of type `str` containing the valid ZIP codes.
1. Construct a `DataFrame` containing only the businesses which DO NOT have valid ZIP codes. (step 4.2.2 below).


**Step 4.2.1**

<!--
BEGIN QUESTION
name: q2ci
points: 1
-->

In [49]:
valid_zips = pd.read_json('data/sf_zipcodes.json')
# valid_zips = bus[bus['postal_code'].isin(valid_zips['zip_codes'])]
# valid_zips.head()
# bus.head()
# print(bus.postal_code[0])
#######################################################
# bus.postal_code.isin(valid_zips)
# valid_zips['zip_codes'] = valid_zips['zip_codes'].astype(str)
# valid_zips = bus[~bus['postal_code'].isin(valid_zips['zip_codes'])]
#######################################################
valid_zips = valid_zips['zip_codes'].astype(str)

In [50]:
grader.check("q4.2.1")

q4.2.1 results: All test cases passed!

After reading the file, we see that the zip codes have been read as integers.

In [51]:
valid_zips.dtype

dtype('O')

This isn't quite what we want! While zip codes are numbers, they are nominal qualitative data, as you hopefully decided in part 2b (and if you didn't time to go fix your answer). As a result, it makes more sense to store them as a string. To do that, we can use the astype function to generate a copy of the pandas series with the astype function as shown below.

In [52]:
valid_zips = valid_zips.astype("string")

In [53]:
type(valid_zips.dtype)

pandas.core.arrays.string_.StringDtype

Now it's time to do step 2. You will probably want to use the `Series.isin` function. For more information on this function see the [the documentation linked in this internet search](https://www.google.com/search?q=series+isin+pandas&rlz=1C1CHBF_enUS910US910&oq=series+isin+pandas&aqs=chrome..69i57l2j69i59j69i60l2j69i65j69i60l2.1252j0j7&sourceid=chrome&ie=UTF-8)

**Step 4.2.2**

<!--
BEGIN QUESTION
name: q2cii
points: 1
-->

In [54]:
# has_valid_zip = ...
invalid_zip_bus = bus[~bus['postal_code'].isin(valid_zips)]
# invalid_zip_bus.head(20)

In [55]:
grader.check("q4.2.2")

q4.2.2 results: All test cases passed!

<br/><br/><br/>

---

#### Part 4.2.3

In the previous question, many of the businesses had a common invalid postal code that was likely used to encode a MISSING postal code.  Do they all share a potentially "interesting address"?

In the following cell, construct a **series** that counts the number of businesses at each `address` that have this single likely MISSING postal code value.  Order the series in descending order by count.


<!--
BEGIN QUESTION
name: q2d
points: 2
-->

In [56]:
missing_df = invalid_zip_bus[invalid_zip_bus['postal_code'] == '-9999']

missing_zip_address_count = missing_df['address'].value_counts()
missing_zip_address_count.head()

Off The Grid                  39
Off the Grid                  10
OTG                            4
Approved Locations             3
Approved Private Locations     3
Name: address, dtype: int64

In [57]:
grader.check("q4.2.3")

q4.2.3 results: All test cases passed!

<!-- END QUESTION -->

<br/><br/><br/>

---

#### Part 4.2.4

Examine the `invalid_zip_bus` dataframe we computed above and look at the businesses that DO NOT have the special MISSING ZIP code value. Some of the invalid postal codes are just the full 9 digit code rather than the first 5 digits. Create a new column named `postal5` in the original `bus` dataframe which contains only the first 5 digits of the `postal_code` column.

Then, for any of the `postal5` ZIP code entries that were not a valid San Francisco ZIP Code (according to `valid_zips`), the provided code will set the `postal5` value to `None`.

**Do not modify the provided code!**

<!--
BEGIN QUESTION
name: q2f
points: 2
-->

In [58]:
bus['postal5'] = bus['postal_code'].apply(lambda x: x[:5])

bus.loc[~bus['postal5'].isin(valid_zips), 'postal5'] = None
# Checking the corrected postal5 column
##################################################################
# README: business id columns was derived to 'bid' in the original 
# notebook, i have changed it to the way it was saved in the dataframe
##################################################################
bus.loc[invalid_zip_bus.index, ['bid', 'name', 'postal_code', 'postal5']]

,bid,name,postal_code,postal5
22,100126,Lamas Peruvian Food Truck,-9999,None
68,100417,"COMPASS ONE, LLC",94105-1420,94105
96,100660,TEAPENTER,94122-1909,94122
109,100781,LE CAFE DU SOLEIL,94117-3504,94117
144,101084,Deli North 200,94518,None
...,...,...,...,...
6173,99369,HOTEL BIRON,94102-5917,94102
6174,99376,Mashallah Halal Food truck Ind,-9999,None
6199,99536,FAITH SANDWICH #2,94105-2907,94105
6204,99681,Twister,95112,None


In [59]:
grader.check("q4.2.4")

q4.2.4 results: All test cases passed!

<br/><br/><br/>

---

<br/><br/><br/>

### Question 5: Investigate the Inspection Data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns named
`iid`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections.

Let's start by looking again at the first 5 rows of `ins` to see what we're working with.

In [60]:
ins.head(5)

,iid,date,score,type
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup


<br/><br/><br/>

---

#### Part 5.1

The column `iid` probably corresponds to an inspection id.  Is it a primary key?  Write an expression (line of code) that evaluates to `True` or `False` based on whether all the values are unique.

**Hint:** This is a very similar question to Question 1a.


<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [61]:
is_ins_iid_a_primary_key = ins['iid'].is_unique
is_ins_iid_a_primary_key

True

In [62]:
grader.check("q5.1")

q5.1 results: All test cases passed!

<br/><br/><br/>

---

#### Part 5.2

Later in this HW, we're going to merge the `bid` and `ins` DataFrames. To do this, we'll need to extract the `bid` from each row. If we look carefully, the column `iid` of the `ins` DataFrame appears to be the composition of two numbers and the first number looks like a business id.

Create a new column called `bid` in the `ins` dataframe containing just the business id.  You will want to use `ins['iid'].str` operations to do this.  Also be sure to convert the type of this column to `int`. Hint: Similar to the early problem where we used `astype(string)` to convert a column to a String, here you should use `astype(int)` to convert the `bid` column into type int.

Optional: Write code which computes the number of `bid` values in `ins` which do not appear in `bus`. In other words, do we have any inspection results for restaurants which do not appear in our business dataset? If so, how many?

**No python `for` loops or list comprehensions are allowed, even for the optional problem.** This is on the honor system since our autograder isn't smart enough to check, but if you're using `for` loops or list comprehensions, you're doing the HW incorrectly.

In [63]:
ins['bid'] = ins['iid'].apply(lambda x: x[:x.index('_')]).astype(int)
# ins.head(5)

In [64]:
grader.check("q5.2")

q5.2 results: All test cases passed!

<br/><br/><br/>

---

#### Part 5.3

For this part, we're going to explore some new somewhat strange syntax that we haven't seen in lecture. Don't panic! If you're not sure what to do, try experimenting, Googling, and don't shy away from talking to other students or course staff.

For this problem we'll use the time component of the inspection data.  All of this information is given in the `date` column of the `ins` dataframe.

**Step 5.3.1:** What is the type of the individual `ins['date']` entries? You may want to grab the very first entry and use the `type` function in python.

**Step 5.3.2:** Rather than the type you discovered in Part 1, we want each entry in `pd.TimeStamp` format. You might expect that the usual way to convert something from it current type to `TimeStamp` would be to use `astype`. You can do that, but the more typical way is to use `pd.to_datetime`. Using `pd.to_datetime`, create a new `ins['timestamp']` column containing `pd.Timestamp` objects.  These will allow us to do date manipulation with much greater ease in part 3 and part 4

**Step 5.3.3:** What are the earliest and latest dates in our inspection data?  *Hint: you can use `min` and `max` on dates of the correct type.*

**Step 5.3.4:** We probably want to examine the inspections by year. Create an additional `ins['year']` column containing just the year of the inspection.  Consider using `pd.Series.dt.year` to do this.

In case you're curious, the documentation for `TimeStamp` data can be found at [this link](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp).

**No python `for` loops or list comprehensions are allowed!**

**Step 5.3.1**

<!--
BEGIN QUESTION
name: q3ci
points: 1
-->

In [65]:
ins_date_type = type(ins['date'][0])
ins_date_type

str

In [66]:
grader.check("q5.3.1")

q5.3.1 results: All test cases passed!

**Step 5.3.2**

<!--
BEGIN QUESTION
name: q3cii
points: 1
-->

In [67]:
ins['timestamp'] = pd.to_datetime(ins['date'])

In [68]:
grader.check("q5.3.2")

q5.3.2 results: All test cases passed!

**Step 5.3.3**

<!--
BEGIN QUESTION
name: q3ciii
points: 1
-->

In [69]:
earliest_date = min(ins['timestamp'])
latest_date = max(ins['timestamp'])

print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)

Earliest Date: 2016-10-04 00:00:00
Latest Date: 2019-11-28 00:00:00


In [70]:
grader.check("q5.3.3")

q5.3.3 results: All test cases passed!

**Step 5.3.4**

<!--
BEGIN QUESTION
name: q3civ
points: 1
-->

In [71]:
ins['year'] = (ins['timestamp']).dt.year

In [72]:
grader.check("q5.3.4")

q5.3.4 results: All test cases passed!

In [73]:
ins.head()

,iid,date,score,type,bid,timestamp,year
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010,2019-03-29,2019
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017,2019-04-17,2019
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017,2019-08-26,2019


<br/><br/><br/>

---

#### Part 5.4

Let's examine the inspection scores `ins['score']`



In [74]:
ins['score'].value_counts().head()

-1      12632
 100     1993
 96      1681
 92      1260
 94      1250
Name: score, dtype: int64

There are a large number of inspections with the `'score'` of `-1`.   These are probably missing values.  Let's see what type of inspections have scores and which do not. Create the following dataframe, and assign it to the variable `ins_missing_score_pivot`. You'll want to use the `pivot_table` method of the DataFrame class, which you can read about in the [pivot_table documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html).

You should observe that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections.


<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Missing Score</th>      <th>False</th>      <th>True</th>      <th>Total</th>    </tr>    <tr>      <th>type</th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Routine - Unscheduled</th>      <td>14031</td>      <td>46</td>      <td>14077</td>    </tr>    <tr>      <th>Reinspection/Followup</th>      <td>0</td>      <td>6439</td>      <td>6439</td>    </tr>    <tr>      <th>New Ownership</th>      <td>0</td>      <td>1592</td>      <td>1592</td>    </tr>    <tr>      <th>Complaint</th>      <td>0</td>      <td>1458</td>      <td>1458</td>    </tr>    <tr>      <th>New Construction</th>      <td>0</td>      <td>994</td>      <td>994</td>    </tr>    <tr>      <th>Non-inspection site visit</th>      <td>0</td>      <td>811</td>      <td>811</td>    </tr>    <tr>      <th>New Ownership - Followup</th>      <td>0</td>      <td>499</td>      <td>499</td>    </tr>    <tr>      <th>Structural Inspection</th>      <td>0</td>      <td>394</td>      <td>394</td>    </tr>    <tr>      <th>Complaint Reinspection/Followup</th>      <td>0</td>      <td>227</td>      <td>227</td>    </tr>    <tr>      <th>Foodborne Illness Investigation</th>      <td>0</td>      <td>115</td>      <td>115</td>    </tr>    <tr>      <th>Routine - Scheduled</th>      <td>0</td>      <td>46</td>      <td>46</td>    </tr>    <tr>      <th>Administrative or Document Review</th>      <td>0</td>      <td>4</td>      <td>4</td>    </tr>    <tr>      <th>Multi-agency Investigation</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    <tr>      <th>Special Event</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    <tr>      <th>Community Health Assessment</th>      <td>0</td>      <td>1</td>      <td>1</td>    </tr>  </tbody></table>

Note that we create a "Missing Score" column, which will be `"True"` for inspections with a missing score, and `"False"` for those with a proper score. This column may be helpful, but you don't need to use it if you don't want to.

In [75]:
type(ins)

pandas.core.frame.DataFrame

In [76]:
ins['Missing Score'] = (ins['score'] == -1).astype("str")

ins_missing_score_pivot = pd.pivot_table(ins, index = 'type', columns = 'Missing Score', aggfunc ={'Missing Score': ['count']}, fill_value = 0) #
ins_missing_score_pivot.columns = ins_missing_score_pivot.columns.get_level_values('Missing Score')
ins_missing_score_pivot['total'] =  ins_missing_score_pivot['False']+ins_missing_score_pivot['True']
ins_missing_score_pivot= ins_missing_score_pivot.sort_values('True', ascending=False).sort_values('False', ascending=False)

display(ins_missing_score_pivot)

Missing Score,False,True,total
type,,,
Routine - Unscheduled,14031,46,14077
Reinspection/Followup,0,6439,6439
New Ownership,0,1592,1592
Complaint,0,1458,1458
New Construction,0,994,994
Non-inspection site visit,0,811,811
New Ownership - Followup,0,499,499
Structural Inspection,0,394,394
Complaint Reinspection/Followup,0,227,227


In [77]:
grader.check("q5.4")

q5.4 results: All test cases passed!

Notice that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections. It is reasonable that for inspection types such as `New Ownership` and `Complaint` to have no associated inspection scores, but we might be curious why there are no inspection scores for the `Reinspection/Followup` inspection type.

<!-- END QUESTION -->



## Congratulations! You have finished Homework 1!

<!-- END QUESTION -->



**Important**: To make sure the test cases run correctly, click `Kernel>Restart & Run All` and make sure all of the test cases are still passing.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [78]:
grader.check_all()

q1.1 results: All test cases passed!

q1.2 results: All test cases passed!

q1.3 results: All test cases passed!

q1.5 results: All test cases passed!

q1.6 results: All test cases passed!

q2.1 results: All test cases passed!

q2.3 results: All test cases passed!

q3.1 results: All test cases passed!

q3.2 results: All test cases passed!

q3.3 results: All test cases passed!

q4.1 results: All test cases passed!

q4.2.1 results: All test cases passed!

q4.2.2 results: All test cases passed!

q4.2.3 results: All test cases passed!

q4.2.4 results: All test cases passed!

q5.1 results: All test cases passed!

q5.2 results: All test cases passed!

q5.3.1 results: All test cases passed!

q5.3.2 results: All test cases passed!

q5.3.3 results: All test cases passed!

q5.3.4 results: All test cases passed!

q5.4 results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [80]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)

ValueError: Expected a notebook file named 'HW01.ipynb' but no such file found